In [1]:
import os
import random
import pickle
from collections import defaultdict, Counter

import numpy as np

from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib inline

from models import *

In [2]:
plt.rcParams["figure.figsize"] = (15, 20)

In [3]:
def read_analogy_data(path):
    with open(path) as reader:
        analogy_data = []
        task_labels = []
        for line in reader:
            if line.startswith(":"):
                task = line.strip().strip(":").strip()
                continue
            # convert to lower-case 
            analogy_data.append(line.strip().lower().split())
            task_labels.append(task)
    return analogy_data, task_labels
analogy_data, task_labels = read_analogy_data("./data/questions-words.txt")

In [4]:
# list of sub-categories
analogy_data[:10], task_labels[:10]

([['athens', 'greece', 'baghdad', 'iraq'],
  ['athens', 'greece', 'bangkok', 'thailand'],
  ['athens', 'greece', 'beijing', 'china'],
  ['athens', 'greece', 'berlin', 'germany'],
  ['athens', 'greece', 'bern', 'switzerland'],
  ['athens', 'greece', 'cairo', 'egypt'],
  ['athens', 'greece', 'canberra', 'australia'],
  ['athens', 'greece', 'hanoi', 'vietnam'],
  ['athens', 'greece', 'havana', 'cuba'],
  ['athens', 'greece', 'helsinki', 'finland']],
 ['capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries',
  'capital-common-countries'])

In [5]:
Counter(task_labels)

Counter({'capital-common-countries': 506,
         'capital-world': 4524,
         'city-in-state': 2467,
         'currency': 866,
         'family': 506,
         'gram1-adjective-to-adverb': 992,
         'gram2-opposite': 812,
         'gram3-comparative': 1332,
         'gram4-superlative': 1122,
         'gram5-present-participle': 1056,
         'gram6-nationality-adjective': 1599,
         'gram7-past-tense': 1560,
         'gram8-plural': 1332,
         'gram9-plural-verbs': 870})

In [6]:
bow2_sim = load_model("bow2.words")
#bow5_sim = load_model("bow5.words")
#deps_sim = load_model("deps.words")

# models = {
#     "bow2": bow2_sim,
#     "bow5": bow5_sim,
#     "deps": deps_sim
# }

In [7]:
def reciprocal_rank(correct_value, results):
    try:
        position = results.index(correct_value)
        return 1 / (position + 1)
    except:
        return 0 

# tests
print(reciprocal_rank("cats", ["catten", "cati", "cats"]))
print(reciprocal_rank("tori", ["catten", "tori", "cats"]))
print(reciprocal_rank("virus", ["virus", "cati", "cats"]))

0.3333333333333333
0.5
1.0


In [8]:
def compute_wv(model, a, a_star, b):
    if a not in model.word_index or a_star not in model.word_index or b not in model.word_index:
        return None
    a, a_star, b = model[a], model[a_star], model[b]
    v = a_star - a
    b_star = b + v
    return b_star

In [9]:
task_labels[0]

'capital-common-countries'

In [10]:
# import threading

# class AnalogyThread(threading.Thread):
#     def __init__(self, task, model, data):
#         threading.Thread.__init__(self)
#         self.task = task
#         self.model = model
#         self.data = data
#         self.task_rr = None
#         self.task_correct = None
    
#     def run(self):
#         print("Running: {}".format(self.task))
#         self.task_rr, self.task_correct = evaluate_model(bow2_sim, analogy_data, self.task)
#         print("Finished: {}".format(self.task))

In [11]:
# threads = []
# for task in np.unique(task_labels):
#     at = AnalogyThread(task, bow2_sim, analogy_data)
#     at.start()
#     threads.append(at)
    
# for thread in threads:
#     thread.join()

In [12]:
# def evaluate_model(model, data, task):
#     task_correct = defaultdict(list)
#     task_rr = defaultdict(list)

#     for index, (tlab, (a, a_star, b, b_star_actual)) in enumerate(zip(task_labels, data)):
#         if tlab != task:
#             continue
            
#         if b_star_actual not in model.word_index:
#             skipped += 1
#             continue
#         b_star = compute_wv(model, a, a_star, b)

#         if b_star is None:
#             skipped += 1
#             continue

#         results = model.most_similar_to_vector(b_star, n=len(model.word_index), score=False)
#         # exclude these
#         results = [r for r in results if r not in {a, a_star, b}]
        
#         if results[0] == b_star_actual:
#             task_correct[tlab].append(1)
#         else:
#             task_correct[tlab].append(0)
        
#         task_rr[tlab].append(reciprocal_rank(b_star_actual, results))
        
#         if index == 2:
#             break
        
#         if index % 100 == 0:
#             print("Task: {}, {}".format(index))
    
#     accuracy = sum(task_correct[task]) / len(task_correct[task])
#     print("\t{}:: Accuracy: {}, MRR: {}".format(task, accuracy, np.mean(task_rr[task])))
    
#     return task_correct, task_rr

In [ ]:
def evaluate_model(model, data):
    overall_correct = []
    overall_rr = []
    task_correct = defaultdict(list)
    task_rr = defaultdict(list)

    for index, (tlab, (a, a_star, b, b_star_actual)) in enumerate(zip(task_labels, data)):
        if b_star_actual not in model.word_index:
            skipped += 1
            continue
        b_star = compute_wv(model, a, a_star, b)

        if b_star is None:
            skipped += 1
            continue

        results = model.most_similar_to_vector(b_star, n=len(model.word_index), score=False)
        # exclude these
        results = [r for r in results if r not in {a, a_star, b}]
        
        if results[0] == b_star_actual:
            overall_correct.append(1)
            task_correct[tlab].append(1)
        else:
            overall_correct.append(0)
            task_correct[tlab].append(0)
        
        overall_rr.append(reciprocal_rank(b_star_actual, results))
        task_rr[tlab].append(reciprocal_rank(b_star_actual, results))

        if index % 100 == 0:
            print(index)
    
    accuracy = sum(overall_correct) / len(overall_correct)
    print("Accuracy: {}, MRR: {}".format(accuracy, np.mean(overall_rr)))
    
    print("Per task: ")
    for task in np.unique(task_labels):
        accuracy = sum(task_correct[task]) / len(task_correct[task])
        print("\t{}:: Accuracy: {}, MRR: {}".format(task, accuracy, np.mean(task_rr[task])))
    
evaluate_model(bow2_sim, analogy_data)

0


In [ ]:


# def compute_scores(data, model, task_labels):
#     skipped = 0
    
#     overall_correct = []
#     overall_reciprocal_ranks = []
    
#     for task_label in np.unique(task_labels):
#         print("Task: ", task_label)
#         word_vectors = []
#         parsed_data = []
        
#         for index, (tlab, (a, a_star, b, b_star_actual)) in enumerate(zip(task_labels, data)):
#             if tlab != task_label:
#                 continue
            
#             # if the final word doesn't exist, then there's no point
#             if b_star_actual not in model.word_index:
#                 skipped += 1
#                 continue

#             if index % 250 == 0:
#                 print("\tPreprocessing: {} of {}".format(index, len(data)))

#             b_star = compute_wv(model, a, a_star, b)
#             if b_star is None:
#                 skipped += 1
#                 continue

#             parsed_data.append((a, a_star, b, b_star_actual))
#             word_vectors.append(b_star)

#         word_vectors = np.array(word_vectors)
#         print("\t\t ... done. Constructed word matrix of size: {}".format(word_vectors.shape))
#         print("\tNow computing similarities!")
    
#         similarity_dict = defaultdict(list)
#         for index, (word, word_index) in enumerate(model.word_index.items()):
#             if index % 100 == 0:
#                 print("\tSimilarity : {} of {}".format(index, len(model.word_index)))
#             word_vector = model.embeddings[word_index]

#             similarities = np.apply_along_axis(lambda _: cosine_similarity(word_vector, _), 1, word_vectors)

#             for idx, tup in enumerate(parsed_data):
#                 similarity_dict[tuple(tup)].append((word, similarities[idx]))

#         correct = defaultdict(list)
#         reciprocal_ranks = defaultdict(list)

#         for tup, task in zip(data, task_labels):
#             if task != task_label:
#                 continue
#             tup = tuple(tup)
#             if tup not in similarity_dict:
#                 continue
#             similarity_list = similarity_dict[tup]
#             similarity_list.sort(key=lambda _: -_[1])
#             if similarity_list[0] == b_star_actual:
#                 correct[task].append(1)
#             else:
#                 correct[task].append(0)
#             b_star_results = [_[0] for _ in similarity_list]
#             reciprocal_ranks[task].append(reciprocal_rank(b_star_actual, b_star_results))

#         overall_correct.extend(correct[task])
#         overall_reciprocal_ranks.extend(reciprocal_ranks[task])
        
#         accuracy = np.sum(correct[task]) / len(correct[task])
#         mrr = np.mean(reciprocal_ranks[task])
#         print("\tTask: {} :: Accuracy: {}, MRR: {}".format(task, accuracy, mrr))        
        
#     overall_acc = np.sum(overall_correct) / len(overall_correct) 
#     print("Overall: Accuracy: {}, MRR: {}".format(overall_acc, np.mean(overall_reciprocal_ranks)))
    
#     print("\t ... done")
#     print("\t Skipped: {}".format(skipped))

# compute_scores(analogy_data, bow2_sim, task_labels)